In [ ]:
import logging
import sys
from ct.aws import get_secret
from ct.utils import plt

root = logging.getLogger()
root.setLevel(logging.ERROR)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
from ct.utils import ts2dt,dt2ts

In [ ]:
import requests
from datetime import datetime as dt
API_KEY = '1wvlrc0JsEMIWF6enI69cWiOojq'

def glassnode(url='https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=dt(2021,8,1),Nretry=10,API_KEY=""):
    """
    i
    1 month (1month)
    1 week (1w)
    1 day (24h)
    1 hour (1h)
    10 minutes (10m)
    
    """
    timestamp=dt2ts(s,delta='1s')
    print(timestamp)
    
    try:
        res = requests.get(url,params={'a': a,'i':i,'s':timestamp,'api_key': API_KEY}) #, 's':1629500000
        df = pd.read_json(res.text, convert_dates=['t']).set_index('t') 
    except Exception as e:
        #print(e,res.text)
        if Nretry==0:
            raise e
        time.sleep(120)
        return glassnode(url=url,a=a,i=i,s=s,Nretry=Nretry-1,API_KEY=API_KEY)
    return df
dfii=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=dt.now()-timedelta(hours=1000),API_KEY=API_KEY)

In [ ]:
dfii.iplot()

In [ ]:
qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='ppttrades',date=[(dt.now()-pd.Timedelta("2D")).date(),dt.today().date()],addwhere=', vol>0.01',verbose=0).iplot()

In [ ]:
dt.now()

In [ ]:
import collections
import json
import ccxt
from ccxt.base.errors import NetworkError, ExchangeError

import backtrader as bt
from functools import wraps

from backtrader import BrokerBase, OrderBase, Order
from backtrader.position import Position
from backtrader.utils.py3 import queue, with_metaclass

from backtrader.metabase import MetaParams
from backtrader.feed import DataBase




from ccxt import DDoSProtection

class MetaCCXTFeed(DataBase.__class__):
    def __init__(cls, name, bases, dct):
        '''Class has already been created ... register'''
        # Initialize the class
        super(MetaCCXTFeed, cls).__init__(name, bases, dct)

        # Register with the store
        CCXTStore.DataCls = cls



class MetaSingleton(MetaParams):
    '''Metaclass to make a metaclassed class a singleton'''

    def __init__(cls, name, bases, dct):
        super(MetaSingleton, cls).__init__(name, bases, dct)
        cls._singleton = None

    def __call__(cls, *args, **kwargs):
        if cls._singleton is None:
            cls._singleton = (
                super(MetaSingleton, cls).__call__(*args, **kwargs))

        return cls._singleton

class CCXTStore(with_metaclass(MetaSingleton, object)):
    '''API provider for CCXT feed and broker classes.
    Added a new get_wallet_balance method. This will allow manual checking of the balance.
        The method will allow setting parameters. Useful for getting margin balances
    Added new private_end_point method to allow using any private non-unified end point
    '''

    # Supported granularities
    _GRANULARITIES = {
        (bt.TimeFrame.Minutes, 1): '1m',
        (bt.TimeFrame.Minutes, 3): '3m',
        (bt.TimeFrame.Minutes, 5): '5m',
        (bt.TimeFrame.Minutes, 15): '15m',
        (bt.TimeFrame.Minutes, 30): '30m',
        (bt.TimeFrame.Minutes, 60): '1h',
        (bt.TimeFrame.Minutes, 90): '90m',
        (bt.TimeFrame.Minutes, 120): '2h',
        (bt.TimeFrame.Minutes, 180): '3h',
        (bt.TimeFrame.Minutes, 240): '4h',
        (bt.TimeFrame.Minutes, 360): '6h',
        (bt.TimeFrame.Minutes, 480): '8h',
        (bt.TimeFrame.Minutes, 720): '12h',
        (bt.TimeFrame.Days, 1): '1d',
        (bt.TimeFrame.Days, 3): '3d',
        (bt.TimeFrame.Weeks, 1): '1w',
        (bt.TimeFrame.Weeks, 2): '2w',
        (bt.TimeFrame.Months, 1): '1M',
        (bt.TimeFrame.Months, 3): '3M',
        (bt.TimeFrame.Months, 6): '6M',
        (bt.TimeFrame.Years, 1): '1y',
    }

    BrokerCls = None  # broker class will auto register
    DataCls = None  # data class will auto register

    @classmethod
    def getdata(cls, *args, **kwargs):
        '''Returns ``DataCls`` with args, kwargs'''
        return cls.DataCls(*args, **kwargs)

    @classmethod
    def getbroker(cls, *args, **kwargs):
        '''Returns broker with *args, **kwargs from registered ``BrokerCls``'''
        return cls.BrokerCls(*args, **kwargs)

    def __init__(self, exchange, currency, config, retries, debug=False, sandbox=False):
        self.exchange = getattr(ccxt, exchange)(config)
        if sandbox:
            self.exchange.set_sandbox_mode(True)
        self.currency = currency
        self.retries = retries
        self.debug = debug
        balance = self.exchange.fetch_balance() if 'secret' in config else 0

        if balance == 0 or not balance['free'][currency]:
            self._cash = 0
        else:
            self._cash = balance['free'][currency]

        if balance == 0 or not balance['total'][currency]:
            self._value = 0
        else:
            self._value = balance['total'][currency]

    def get_granularity(self, timeframe, compression):
        if not self.exchange.has['fetchOHLCV']:
            raise NotImplementedError("'%s' exchange doesn't support fetching OHLCV data" % \
                                      self.exchange.name)

        granularity = self._GRANULARITIES.get((timeframe, compression))
        if granularity is None:
            raise ValueError("backtrader CCXT module doesn't support fetching OHLCV "
                             "data for time frame %s, comression %s" % \
                             (bt.TimeFrame.getname(timeframe), compression))

        if self.exchange.timeframes and granularity not in self.exchange.timeframes:
            raise ValueError("'%s' exchange doesn't support fetching OHLCV data for "
                             "%s time frame" % (self.exchange.name, granularity))

        return granularity

    def retry(method):
        @wraps(method)
        def retry_method(self, *args, **kwargs):
            for i in range(self.retries):
                if self.debug:
                    print('{} - {} - Attempt {}'.format(datetime.now(), method.__name__, i))
                time.sleep(self.exchange.rateLimit / 5) #10seconds for binance
                print(f"retry ratelimit={self.exchange.rateLimit}")
                try:
                    return method(self, *args, **kwargs)
                except (NetworkError, ExchangeError):
                    if i == self.retries - 1:
                        raise

        return retry_method

    @retry
    def get_wallet_balance(self, currency, params=None):
        balance = self.exchange.fetch_balance(params)
        return balance

    @retry
    def get_balance(self):
        balance = self.exchange.fetch_balance()

        cash = balance['free'][self.currency]
        value = balance['total'][self.currency]
        # Fix if None is returned
        self._cash = cash if cash else 0
        self._value = value if value else 0

    @retry
    def getposition(self):
        return self._value
        # return self.getvalue(currency)

    @retry
    def create_order(self, symbol, order_type, side, amount, price, params):
        # returns the order
        return self.exchange.create_order(symbol=symbol, type=order_type, side=side,
                                          amount=amount, price=price, params=params)

    @retry
    def cancel_order(self, order_id, symbol):
        return self.exchange.cancel_order(order_id, symbol)

    @retry
    def fetch_trades(self, symbol):
        return self.exchange.fetch_trades(symbol)

    @retry
    def fetch_ohlcv(self, symbol, timeframe, since, limit, params={}):
        if self.debug:
            print('Fetching: {}, TF: {}, Since: {}, Limit: {}'.format(symbol, timeframe, since, limit))
        return self.exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit, params=params)

    @retry
    def fetch_order(self, oid, symbol):
        return self.exchange.fetch_order(oid, symbol)

    @retry
    def fetch_open_orders(self,**kwargs):
        return self.exchange.fetchOpenOrders(**kwargs)

    @retry
    def private_end_point(self, type, endpoint, params):
        '''
        Open method to allow calls to be made to any private end point.
        See here: https://github.com/ccxt/ccxt/wiki/Manual#implicit-api-methods
        - type: String, 'Get', 'Post','Put' or 'Delete'.
        - endpoint = String containing the endpoint address eg. 'order/{id}/cancel'
        - Params: Dict: An implicit method takes a dictionary of parameters, sends
          the request to the exchange and returns an exchange-specific JSON
          result from the API as is, unparsed.
        To get a list of all available methods with an exchange instance,
        including implicit methods and unified methods you can simply do the
        following:
        print(dir(ccxt.hitbtc()))
        '''
        return getattr(self.exchange, endpoint)(params)

class CCXTOrder(OrderBase):
    def __init__(self, owner, data, ccxt_order):
        self.owner = owner
        self.data = data
        self.ccxt_order = ccxt_order
        self.executed_fills = []
        self.ordtype = self.Buy if ccxt_order['side'] == 'buy' else self.Sell
        self.size = float(ccxt_order['amount'])

        super(CCXTOrder, self).__init__()


class MetaCCXTBroker(BrokerBase.__class__):
    def __init__(cls, name, bases, dct):
        '''Class has already been created ... register'''
        # Initialize the class
        super(MetaCCXTBroker, cls).__init__(name, bases, dct)
        CCXTStore.BrokerCls = cls


class CCXTBroker(with_metaclass(MetaCCXTBroker, BrokerBase)):
    '''Broker implementation for CCXT cryptocurrency trading library.
    This class maps the orders/positions from CCXT to the
    internal API of ``backtrader``.
    Broker mapping added as I noticed that there differences between the expected
    order_types and retuned status's from canceling an order
    Added a new mappings parameter to the script with defaults.
    Added a get_balance function. Manually check the account balance and update brokers
    self.cash and self.value. This helps alleviate rate limit issues.
    Added a new get_wallet_balance method. This will allow manual checking of the any coins
        The method will allow setting parameters. Useful for dealing with multiple assets
    Modified getcash() and getvalue():
        Backtrader will call getcash and getvalue before and after next, slowing things down
        with rest calls. As such, th
    The broker mapping should contain a new dict for order_types and mappings like below:
    broker_mapping = {
        'order_types': {
            bt.Order.Market: 'market',
            bt.Order.Limit: 'limit',
            bt.Order.Stop: 'stop-loss', #stop-loss for kraken, stop for bitmex
            bt.Order.StopLimit: 'stop limit'
        },
        'mappings':{
            'closed_order':{
                'key': 'status',
                'value':'closed'
                },
            'canceled_order':{
                'key': 'result',
                'value':1}
                }
        }
    Added new private_end_point method to allow using any private non-unified end point
    '''

    order_types = {Order.Market: 'market',
                   Order.Limit: 'limit',
                   Order.Stop: 'stop',  # stop-loss for kraken, stop for bitmex
                   Order.StopLimit: 'stop limit'}

    mappings = {
        'closed_order': {
            'key': 'status',
            'value': 'closed'
        },
        'canceled_order': {
            'key': 'status',
            'value': 'canceled'}
    }

    def __init__(self, broker_mapping=None, debug=False, **kwargs):
        super(CCXTBroker, self).__init__()

        if broker_mapping is not None:
            try:
                self.order_types = broker_mapping['order_types']
            except KeyError:  # Might not want to change the order types
                pass
            try:
                self.mappings = broker_mapping['mappings']
            except KeyError:  # might not want to change the mappings
                pass

        self.store = CCXTStore(**kwargs)

        self.currency = self.store.currency

        self.positions = collections.defaultdict(Position)

        self.debug = debug
        self.indent = 4  # For pretty printing dictionaries

        self.notifs = queue.Queue()  # holds orders which are notified

        self.open_orders = list()

        self.startingcash = self.store._cash
        self.startingvalue = self.store._value

    def get_balance(self):
        self.store.get_balance()
        self.cash = self.store._cash
        self.value = self.store._value
        return self.cash, self.value

    def get_wallet_balance(self, currency, params={}):
        balance = self.store.get_wallet_balance(currency, params=params)
        cash = balance['free'][currency] if balance['free'][currency] else 0
        value = balance['total'][currency] if balance['total'][currency] else 0
        return cash, value

    def getcash(self):
        # Get cash seems to always be called before get value
        # Therefore it makes sense to add getbalance here.
        # return self.store.getcash(self.currency)
        self.cash = self.store._cash
        return self.cash

    def getvalue(self, datas=None):
        # return self.store.getvalue(self.currency)
        self.value = self.store._value
        return self.value

    def get_notification(self):
        try:
            return self.notifs.get(False)
        except queue.Empty:
            return None

    def notify(self, order):
        self.notifs.put(order)

    def getposition(self, data, clone=True):
        # return self.o.getposition(data._dataname, clone=clone)
        pos = self.positions[data._dataname]
        if clone:
            pos = pos.clone()
        return pos

    def next(self):
        if self.debug:
            print('Broker next() called')
        
        for o_order in list(self.open_orders):
            oID = o_order.ccxt_order['id']

            # Print debug before fetching so we know which order is giving an
            # issue if it crashes
            if self.debug:
                print('Fetching Order ID: {}'.format(oID))

            # Get the order
            ccxt_order = self.store.fetch_order(oID, o_order.data.p.dataname)
            
            # Check for new fills
            if ccxt_order is not None and 'trades' in ccxt_order and ccxt_order['trades'] is not None:
                print(f'ccxt_order={ccxt_order}')
                for fill in ccxt_order['trades']:
                    if fill not in o_order.executed_fills:
                        o_order.execute(fill['datetime'], fill['amount'], fill['price'], 
                                        0, 0.0, 0.0, 
                                        0, 0.0, 0.0, 
                                        0.0, 0.0,
                                        0, 0.0)
                        o_order.executed_fills.append(fill['id'])

            if self.debug:
                print(json.dumps(ccxt_order, indent=self.indent))

            # Check if the order is closed
            if ccxt_order[self.mappings['closed_order']['key']] == self.mappings['closed_order']['value']:
                pos = self.getposition(o_order.data, clone=False)
                pos.update(o_order.size, o_order.price)
                o_order.completed()
                self.notify(o_order)
                self.open_orders.remove(o_order)
                self.get_balance()

    def _submit(self, owner, data, exectype, side, amount, price, params):
        order_type = self.order_types.get(exectype) if exectype else 'market'
        created = int(data.datetime.datetime(0).timestamp()*1000)
        # Extract CCXT specific params if passed to the order
        params = params['params'] if 'params' in params else params
        params['created'] = created  # Add timestamp of order creation for backtesting
        ret_ord = self.store.create_order(symbol=data.p.dataname, order_type=order_type, side=side,
                                          amount=amount, price=price, params=params)

        _order = self.store.fetch_order(ret_ord['id'], data.p.dataname)

        order = CCXTOrder(owner, data, _order)
        order.price = ret_ord['price']
        self.open_orders.append(order)

        self.notify(order)
        return order

    def buy(self, owner, data, size, price=None, plimit=None,
            exectype=None, valid=None, tradeid=0, oco=None,
            trailamount=None, trailpercent=None,
            **kwargs):
        del kwargs['parent']
        del kwargs['transmit']
        return self._submit(owner, data, exectype, 'buy', size, price, kwargs)

    def sell(self, owner, data, size, price=None, plimit=None,
             exectype=None, valid=None, tradeid=0, oco=None,
             trailamount=None, trailpercent=None,
             **kwargs):
        del kwargs['parent']
        del kwargs['transmit']
        return self._submit(owner, data, exectype, 'sell', size, price, kwargs)

    def cancel(self, order):

        oID = order.ccxt_order['id']

        if self.debug:
            print('Broker cancel() called')
            print('Fetching Order ID: {}'.format(oID))

        # check first if the order has already been filled otherwise an error
        # might be raised if we try to cancel an order that is not open.
        ccxt_order = self.store.fetch_order(oID, order.data.p.dataname)

        if self.debug:
            print(json.dumps(ccxt_order, indent=self.indent))

        if ccxt_order[self.mappings['closed_order']['key']] == self.mappings['closed_order']['value']:
            return order

        ccxt_order = self.store.cancel_order(oID, order.data.p.dataname)

        if self.debug:
            print(json.dumps(ccxt_order, indent=self.indent))
            print('Value Received: {}'.format(ccxt_order[self.mappings['canceled_order']['key']]))
            print('Value Expected: {}'.format(self.mappings['canceled_order']['value']))

        if ccxt_order[self.mappings['canceled_order']['key']] == self.mappings['canceled_order']['value']:
            self.open_orders.remove(order)
            order.cancel()
            self.notify(order)
        return order

    def get_orders_open(self, safe=False):
        return self.store.fetch_open_orders()

    def private_end_point(self, type, endpoint, params):
        '''
        Open method to allow calls to be made to any private end point.
        See here: https://github.com/ccxt/ccxt/wiki/Manual#implicit-api-methods
        - type: String, 'Get', 'Post','Put' or 'Delete'.
        - endpoint = String containing the endpoint address eg. 'order/{id}/cancel'
        - Params: Dict: An implicit method takes a dictionary of parameters, sends
          the request to the exchange and returns an exchange-specific JSON
          result from the API as is, unparsed.
        To get a list of all available methods with an exchange instance,
        including implicit methods and unified methods you can simply do the
        following:
        print(dir(ccxt.hitbtc()))
        '''
        endpoint_str = endpoint.replace('/', '_')
        endpoint_str = endpoint_str.replace('{', '')
        endpoint_str = endpoint_str.replace('}', '')

        method_str = 'private_' + type.lower() + endpoint_str.lower()

        return self.store.private_end_point(type=type, endpoint=method_str, params=params)  
    
    
    
    
import time
from collections import deque
from datetime import datetime

import backtrader as bt
from backtrader.feed import DataBase
from backtrader.utils.py3 import with_metaclass



class CCXTFeed(with_metaclass(MetaCCXTFeed, DataBase)):
    """
    CryptoCurrency eXchange Trading Library Data Feed.
    Params:
      - ``historical`` (default: ``False``)
        If set to ``True`` the data feed will stop after doing the first
        download of data.
        The standard data feed parameters ``fromdate`` and ``todate`` will be
        used as reference.
      - ``backfill_start`` (default: ``True``)
        Perform backfilling at the start. The maximum possible historical data
        will be fetched in a single request.
    Changes From Ed's pacakge
        - Added option to send some additional fetch_ohlcv_params. Some exchanges (e.g Bitmex)
          support sending some additional fetch parameters.
        - Added drop_newest option to avoid loading incomplete candles where exchanges
          do not support sending ohlcv params to prevent returning partial data
    """

    params = (
        ('historical', False),  # only historical download
        ('backfill_start', False),  # do backfilling at the start
        ('fetch_ohlcv_params', {}),
        ('ohlcv_limit', 20),
        ('drop_newest', False),
        ('debug', False)
    )

    _store = CCXTStore

    # States for the Finite State Machine in _load
    _ST_LIVE, _ST_HISTORBACK, _ST_OVER = range(3)

    # def __init__(self, exchange, symbol, ohlcv_limit=None, config={}, retries=5):
    def __init__(self, **kwargs):
        # self.store = CCXTStore(exchange, config, retries)
        self.store = self._store(**kwargs)
        self._data = deque()  # data queue for price data
        self._last_id = ''  # last processed trade id for ohlcv
        self._last_ts = 0  # last processed timestamp for ohlcv

    def start(self, ):
        DataBase.start(self)

        if self.p.fromdate:
            self._state = self._ST_HISTORBACK
            self.put_notification(self.DELAYED)
            self._fetch_ohlcv(self.p.fromdate)

        else:
            self._state = self._ST_LIVE
            self.put_notification(self.LIVE)

    def _load(self):
        
        
        #if np.random.randint(100)==5:
        #print(f" _load()  state={self._state} data len {len(self._data)}" )
        
        
        if self._state == self._ST_OVER:
            return False

        while True:
            if self._state == self._ST_LIVE:
                if self._timeframe == bt.TimeFrame.Ticks:
                    return self._load_ticks()
                else:
                    
                    for i in range(5):
                        try:
                            self._fetch_ohlcv()
                            break
                        except DDoSProtection:
                            print(f"DDOS binance!..sleeping 5min  dt.now()= {dt.now()}")
                            time.sleep(5*60)
                            

                    ret = self._load_ohlcv()
                    if self.p.debug:
                        print('----     LOAD    ----')
                        print('{} Load OHLCV Returning: {}'.format(datetime.utcnow(), ret))
                    return ret

            elif self._state == self._ST_HISTORBACK:
                ret = self._load_ohlcv()
                if ret:
                    return ret
                else:
                    # End of historical data
                    if self.p.historical:  # only historical
                        self.put_notification(self.DISCONNECTED)
                        self._state = self._ST_OVER
                        return False  # end of historical
                    else:
                        self._state = self._ST_LIVE
                        self.put_notification(self.LIVE)
                        continue

    def _fetch_ohlcv(self, fromdate=None):
        """Fetch OHLCV data into self._data queue"""
        
        #print(f"fetch lastts={self._last_ts} fromdate={self.p.fromdate} todate={self.p.todate}")
        granularity = self.store.get_granularity(self._timeframe, self._compression)

        if fromdate:
            since = int((fromdate - datetime(1970, 1, 1)).total_seconds() * 1000)
        else:
            if self._last_ts > 0:
                since = self._last_ts
            else:
                since = None

        limit = self.p.ohlcv_limit

        while True:
            dlen = len(self._data)

            if self.p.debug:
                # TESTING
                since_dt = datetime.utcfromtimestamp(since // 1000) if since is not None else 'NA'
                print('---- NEW REQUEST ----')
                print('{} - Requesting: Since TS {} Since date {} granularity {}, limit {}, params'.format(
                    datetime.utcnow(), since, since_dt, granularity, limit, self.p.fetch_ohlcv_params))
                data = sorted(self.store.fetch_ohlcv(self.p.dataname, timeframe=granularity,
                                                     since=since, limit=limit, params=self.p.fetch_ohlcv_params))
                try:
                    for i, ohlcv in enumerate(data):
                        tstamp, open_, high, low, close, volume = ohlcv
                        print('{} - Data {}: {} - TS {} Time {}'.format(datetime.utcnow(), i,
                                                                        datetime.utcfromtimestamp(tstamp // 1000),
                                                                        tstamp, (time.time() * 1000)))
                        # ------------------------------------------------------------------
                except IndexError:
                    print('Index Error: Data = {}'.format(data))
                print('---- REQUEST END ----')
            else:
                start = dt.now()
                data = sorted(self.store.fetch_ohlcv(self.p.dataname, timeframe=granularity,since=since, limit=limit, params=self.p.fetch_ohlcv_params))
                end = datetime.now()
                #print(f"fetchohcv ccxt {end-start}")

            # Check to see if dropping the latest candle will help with
            # exchanges which return partial data
            if self.p.drop_newest:
                del data[-1]

            for ohlcv in data:

                # for ohlcv in sorted(self.store.fetch_ohlcv(self.p.dataname, timeframe=granularity,
                #                                           since=since, limit=limit, params=self.p.fetch_ohlcv_params)):

                if None in ohlcv:
                    continue

                tstamp = ohlcv[0]

                # Prevent from loading incomplete data
                # if tstamp > (time.time() * 1000):
                #    continue

                if tstamp > self._last_ts:
                    if self.p.debug:
                        print('Adding: {}'.format(ohlcv))
                    self._data.append(ohlcv)
                    self._last_ts = tstamp

            if dlen == len(self._data):
                break

    def _load_ticks(self):
        if self._last_id is None:
            # first time get the latest trade only
            trades = [self.store.fetch_trades(self.p.dataname)[-1]]
        else:
            trades = self.store.fetch_trades(self.p.dataname)

        for trade in trades:
            trade_id = trade['id']

            if trade_id > self._last_id:
                trade_time = datetime.strptime(trade['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ')
                self._data.append((trade_time, float(trade['price']), float(trade['amount'])))
                self._last_id = trade_id

        try:
            trade = self._data.popleft()
        except IndexError:
            return None  # no data in the queue

        trade_time, price, size = trade

        self.lines.datetime[0] = bt.date2num(trade_time)
        self.lines.open[0] = price
        self.lines.high[0] = price
        self.lines.low[0] = price
        self.lines.close[0] = price
        self.lines.volume[0] = size

        return True

    def _load_ohlcv(self):
        #print(f"_load_ohlcv state={self._state} data len {len(self._data)}")
        try:
            ohlcv = self._data.popleft()
        except IndexError:
            return None  # no data in the queue

        tstamp, open_, high, low, close, volume = ohlcv

        dtime = datetime.utcfromtimestamp(tstamp // 1000)

        self.lines.datetime[0] = bt.date2num(dtime)
        self.lines.open[0] = open_
        self.lines.high[0] = high
        self.lines.low[0] = low
        self.lines.close[0] = close
        self.lines.volume[0] = volume

        return True

    def haslivedata(self):
        res= (self._state == self._ST_LIVE and self._data)
        #print(f"haslivedata() res = {res}")
        
        
        return res

    def islive(self):
        return not self.p.historical
    
    
    






In [ ]:
import backtrader as bt
from ct.utils import DF,npa,np,pd
import ipdb
import time
from datetime import timedelta
from datetime import datetime as dt

#from ccxtbt import CCXTStore
from ct.utils import qbars,date
#import getpass
from backtrader_plotting.schemes import Blackly
from backtrader_plotting.schemes import Tradimo
from backtrader_plotting.bokeh.live.plotlistener import PlotListener
from ct.alfafactory import trade_list,WalletObserver,SumWalletObserver,CloseObserver,PositionObserver,printTradeAnalysis
from ct.fs import addbb2
# %matplotlib inline

In [ ]:
#print(pd.Series(pd.date_range(dt.now(), periods=1, freq='5S')))
print(pd.Series([dt.now()]).dt.round('5S')[0])

In [ ]:
from backtrader.utils import AutoOrderedDict

class MyTradesAnalyzer(bt.Analyzer):
    params = (
        ('fund', None),
    )

    def start(self):
        super(MyTradesAnalyzer, self).start()

    def create_analysis(self):
        self.rets = AutoOrderedDict()  # dict with . notation
        self.rets.len = 0
        self.rets.nah = 'ole'

    def stop(self):
        self.rets._close()  # . notation cannot create more keys
        
    def notify_trade(self, trade):
        print(f"Mytradeanalyzer:notify_trade() trade={trade}")
        r = self.rets 
        r.drawdown = 1 #call binance get new trade if there is new trade
        r.max.drawdown = 1
        r.len = r.len + 1 
        r.max.len = max(r.max.len, r.len)

    
#     def next(self):
#         r = self.rets
#         r.moneydown = 1
#         r.drawdown = 1
#         r.max.moneydown = 1
#         r.max.drawdown = 1
#         r.len = r.len + 1 
#         r.max.len = max(r.max.len, r.len)

class PropertyObserver(bt.Observer):
    _stclock = True

    params = (
        ('name', 'ole'),
    )

    lines = ( 'value',)
    plotinfo = dict(plot=True, subplot=True)

    def next(self):
        self.lines.value[0] = getattr(self._owner,self.p.name)

        
class DummyIndicator(bt.indicators.Indicator):
    plotinfo = dict(subplot=True)
    lines=('lastliq',)
    
    
class MyBuySell(bt.observers.BuySell):
    plotinfo = dict(plot=True, subplot=True, plotlinelabels=True)
    
    plotlines = dict(
        buy=dict(marker='^', markersize=8.0, color='lime',
                 fillstyle='full', ls=''),
        sell=dict(marker='v', markersize=8.0, color='red',
                  fillstyle='full', ls='')
    )


In [ ]:
1

In [ ]:

PRINTCANDLE=True
class GlassnodeStrategy(bt.Strategy):
    params = (('betsize',0.001),('lags', 20),('delay',4),('trailpercent', 3),('inflowlimit',5),('inflowlimit2',12),('nstd',2))# ('entryz', 1.2), ('exitz',1.2),('methodma','ewm'),('methodstd','ewm'),('trailpercent', 2),('liqlimit',35))
    
    def notify_order(self, order): 
        if not order.alive():
            self.order = None  # No pending orders
        logging.critical(f"|notify_order|{order}|position={self.position.size}|brokerpos={self.broker.getposition(self.datas[0]).size}|c={self.data0.close[0]}|o={self.data0.open[0]}|order={order} ")
         #The logic in the strategy has to remember when a position was entered, probably in notify_order by checking the date from the notified order
        #order.executed.dt which can be translated to a datetime instance like this for example: order.data.num2date(order.executed.dt)`
        #And then checking when the data hits a month later self.data.datetime, where you can get instances with self.data.datetime.datetime() for example.
    def __init__(self):
        print("strat init()")
        self.order = None
        #self.lastliq=DummyIndicator()
         
        
    def start(self):
        print("strat start()")
        self.order = None  # Avoid operrations on pending order

    def next(self):
        if not BACKTEST:
            if self.live_data==False:
                return
        if self.order:
            return  # pending order execution

        curdt=bt.num2date(self.data0.datetime[0])
        o,h,l,c,v=self.data0.open[0],self.data0.high[0],self.data0.low[0],self.data0.close[0],self.data0.volume[0]
        if PRINTCANDLE:print(f"curdt={curdt} c={c}")
        #dfpn=dfcq['btc-all_exchange-exchange-flows-netflow']['netflow_total']#.loc[:curdt]#dfcq['']#qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
        
        
        dfpn=dfcq['btc-all_exchange-exchange-flows-inflow']['inflow_mean']#.loc[:curdt]#dfcq['']#qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
        
        if BACKTEST:
            dfpn=dfgsi#[::-1]
            dfp=dfpn[dfpn.index<curdt].head(self.p.lags)['v']
        else:
            dfp=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=dt.now()-timedelta(hours=2),API_KEY=API_KEY)['v']
            
        

        #ipdb.set_trace()
        
        #display(dfp)
        #display(dfn)

#        df5m=df5m.resample("10Min").last()#10Min?! 5Min?
        if PRINTCANDLE:print(f"\n {dt.now()} next() self.dt={self.datetime.datetime(0)} data0close={self.data0.close[0]} data1close={self.data1.close[0] if hasattr(self,'data1') else 'not present'}  data1close={self.data2.close[0] if hasattr(self,'data2') else 'not present'}  \n")    
        if PRINTCANDLE:print(f"data0 get5={self.data0.close.get(size=5) if hasattr(self,'data0') else 'not present'}")

        if self.position.size==0: # not in the market
#             if len(dfp) and dfp.iloc[0]>dfp.mean():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
#                 logging.critical(f"dfp={dfp}")
#                 self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
#                 logging.critical(f"{self.order}")
#                 self.order = None
            #display(dfn)
            #print(f"dfn.iloc[0]<dfn.mean(){dfn.iloc[0]}<{dfn.mean()}")
            if PRINTCANDLE:print(f"dfp.iloc[0] {dfp.iloc[0]}")
            #if len(dfn) and dfn.iloc[0]<dfn.mean()-self.p.nstd*dfn.std():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
            #ipdb.set_trace()
            
            ##iloc==4 cuz 40 minutes delay
            if BACKTEST:
                condition=(len(dfp)>5 and dfp.iloc[self.p.delay]>self.p.inflowlimit and dfp.iloc[self.p.delay]<self.p.inflowlimit2)
            else:
                display(dfp)
                condition=(dfp.iloc[-1]>self.p.inflowlimit and dfp.iloc[-1]<self.p.inflowlimit2)
                
                    
            if condition :#dfn.mean()-self.p.nstd*dfn.std():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
            # and  momentum is not strong   use rsi from bt?
            #display(dfn)
                #self.order = self.buy(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                
                logging.critical(f"{self.order}")
                self.order = None

        elif self.order is None: # Position in Market
            #self.order = self.sell(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            self.order = self.buy(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            
            logging.critical(f"{self.order}")
            #close_position = binance.create_order(symbol=symbol, type="MARKET", side="buy", amount=pos['positionAmt'], params={"reduceOnly": True}) 

    def notify_data(self, data, status, *args, **kwargs):
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False


    def log(self,txt,dummy=True):
        logging.debug(txt)

In [ ]:
#from ct.alfafactory import 
#from config import ENV, PRODUCTION

        
class MrStrategy(bt.Strategy):
    params = (('betsize',0.001),('lags', 20), ('entryz', 1.2), ('exitz',1.2),('methodma','ewm'),('methodstd','ewm'),('trailpercent', 2),('liqlimit',35))

    
    def notify_order(self, order): 
        if not order.alive():
            self.order = None  # No pending orders
        logging.critical(f"|notify_order|{order}|position={self.position.size}|brokerpos={self.broker.getposition(self.datas[0]).size}|c={self.data0.close[0]}|o={self.data0.open[0]}|order={order} ")


    def __init__(self):
        print("strat init()")
        self.order = None
        self.lastliq=DummyIndicator()
         
        
    def start(self):
        print("strat start()")
        self.order = None  # Avoid operrations on pending order

    def next(self):
        if not BACKTEST:
            if self.live_data==False:
                return
        if self.order:
            return  # pending order execution

        curdt=bt.num2date(self.data0.datetime[0])
        o,h,l,c,v=self.data0.open[0],self.data0.high[0],self.data0.low[0],self.data0.close[0],self.data0.volume[0]
        
        #ipdb.set_trace()
        #indicator
        start= dt.now()
        df1M=qbars(sym='BTCUSDT@binance',freq='1D',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("30D")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
        end= dt.now()
        #print(f"30D time {end-start}")
        #display(df1M)
        
        if not BACKTEST:        
            df5m=qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
            if len(df5m)==0:
                df5m=DF({'o':np.nan,'h':np.nan,'l':np.nan,'c':np.nan,'v':np.nan},index=pd.date_range((curdt-pd.Timedelta("10M")).date(),dt.today().date(),freq='10M'))
            #df5m.loc[dt.now()]=df1M.iloc[0]*np.nan #if empty, use 1M df
            df5m=df5m.resample("10Min").last()#10Min?! 5Min?
        elif BACKTEST:
            df5m=qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),curdt.date()],addwhere=', side=`sell , vol>0.01',verbose=0)
            df5m=df5m[df5m.index<curdt]
            if len(df5m)==0:
                df5m=DF({'o':np.nan,'h':np.nan,'l':np.nan,'c':np.nan,'v':np.nan},index=pd.date_range((curdt-pd.Timedelta("10M")).date(),dt.today().date(),freq='10M'))
            #df5m.loc[dt.now()]=df1M.iloc[0]*np.nan #if empty, use 1M df
            df5m=df5m.resample("10Min").last()#10Min?! 5Min?

        
        self.lastliq=df5m.iloc[-1]['v'] if len(df5m) else 0
        
        #display(df5m)
        
        
        print(f"\n {dt.now()} next() self.dt={self.datetime.datetime(0)} data0close={self.data0.close[0]} data1close={self.data1.close[0] if hasattr(self,'data1') else 'not present'}  data1close={self.data2.close[0] if hasattr(self,'data2') else 'not present'}  \n")    
        #print(f"self.data0.datetime[0]={bt.num2date(self.data0.datetime[0]) if hasattr(self,'data0') else 'not present'}")
#         print(f"self.data1.datetime[0]={bt.num2date(self.data1.datetime[0]) if hasattr(self,'data1') else 'not present'}")
#         print(f"self.data2.datetime[0]={bt.num2date(self.data2.datetime[0]) if hasattr(self,'data2') else 'not present'}")
        
        
        print(f"data0 get5={self.data0.close.get(size=5) if hasattr(self,'data0') else 'not present'}")
#         print(f"data1 get5={self.data1.close.get(size=5) if hasattr(self,'data1') else 'not present'}")
#         print(f"data2 get5={self.data2.close.get(size=5) if hasattr(self,'data1') else 'not present'}")


        if self.position.size==0: # not in the market
            if len(df5m) and df5m.iloc[-1]['v'] >self.p.liqlimit :# and self.data1.reserve<10000000:
                logging.critical(f"df5m={df5m}")
                self.order = self.buy(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                logging.critical(f"{self.order}")
                self.order = None

        elif self.order is None: # Position in Market
            self.order = self.sell(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            logging.critical(f"{self.order}")
            #close_position = binance.create_order(symbol=symbol, type="MARKET", side="buy", amount=pos['positionAmt'], params={"reduceOnly": True}) 

    def notify_data(self, data, status, *args, **kwargs):
#        dn = data._name
#        dt = dt.now()
#        msg= 'Data Status: {}, Order Status: {}'.format(data._getstatusname(status), status)
#        print(dt,dn,msg)
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False


    def log(self,txt,dummy=True):
        logging.debug(txt)


In [ ]:
COIN_TARGET = "BTC"
COIN_REFER = "USDT"

DEBUG = True
BACKTEST=True

LAGS=3
MINUTES=5

def main():
    cerebro = bt.Cerebro(stdstats=False)#quicknotify=True)
    #cerebro.addstrategy(MrStrategy,liqlimit=10)
    cerebro.addstrategy(GlassnodeStrategy,betsize=0.002,lags=20,delay=4,trailpercent=3,inflowlimit=5,inflowlimit2=12)

    #     cerebro.addobserver(bt.observers.DrawDown)
    cerebro.addobserver(bt.observers.Trades)
    cerebro.addobserver(bt.observers.BuySell,barplot=True,bardist=0.02)
#    cerebro.addobserver(MyBuySell,barplot=True,bardist=0.02)
    
#     cerebro.addobserver(bt.observers.Broker)
#     cerebro.addobserver(bt.observers.TimeReturn,fund=False)
#     cerebro.addobserver(bt.observers.Broker)
#     cerebro.addobserver(bt.observers.FundValue)
#     cerebro.addobserver(bt.observers.FundShares)
#    cerebro.addobserver(bt.observers.LogReturns,fund=True)
    

#     cerebro.addobserver(SumWalletObserver,ccys=['USDT','USDC'])
#     cerebro.addobserver(CloseObserver)#,ccys=['USDT','USDC'])
#,'USDC'])
#   cerebro.addobserver(bt.observers.Broker)

#    cerebro.addanalyzer(bt.analyzers.DrawDown)
#    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Days)
#, _name='txn')
    cerebro.addanalyzer(bt.analyzers.TimeReturn)#,_name='timereturn')
#    cerebro.addanalyzer(bt.analyzers.Returns)#,_name='returns')
    #data1=KDBFeed(sym='BTCUSDT@binance',freq="1Min",table='ppttrades',fromdate=dt.today().date(),todate=None,addwhere=', side=`sell , vol>0.01',timeframe=bt.TimeFrame.Minutes ,compression=1)
    #cerebro.adddata(data1)
    
    if BACKTEST is False: 
        cerebro.addobserver(WalletObserver,ccy='USDT')
        cerebro.addobserver(PositionObserver,ccys=['USDT'])

        cerebro.addanalyzer(MyTradesAnalyzer)#,_name='timereturn')
        #cerebro.addobserver(PropertyObserver,name='lastliq')
        cerebro.addlistener(bt.listeners.RecorderListener)
        cerebro.addlistener(PlotListener, volume=False, scheme=Tradimo(hovertool_timeformat='%F %R:%S', tabs='single'), lookback=480,http_port=50004,address='0.0.0.0',debug=True)
        srt=get_secret('b1-wzxNSw')
        broker_config = { 'apiKey': srt['k'],'secret': srt['s'], 'nonce': lambda: str(int(time.time() * 1000)), 'enableRateLimit': True,'options': {'defaultType': 'future'}, }
        store = CCXTStore(exchange='binance', currency=COIN_REFER, config=broker_config, retries=5, debug=False)
        
#        NOTE: Broker mappings are not required if the broker uses the same values as the defaults in CCXTBroker.
#symbol = 'BTC/USDT'
# side = 'sell'
# amount = 0.001
# order_type = 'TRAILING_STOP_MARKET'
# rate = '0.2'
# price = None
# params = {
#     'stopPrice': 17200,
#     'callbackRate': rate
# } order = exchange.create_order(symbol, order_type, side, amount, price, params)
# {
#     "clientOrderId": "testOrder",
#     "cumQty": "0",
#     "cumQuote": "0",
#     "executedQty": "0",
#     "orderId": 22542179,
#     "avgPrice": "0.00000",
#     "origQty": "10",
#     "price": "0",
#     "reduceOnly": false,
#     "side": "BUY",
#     "positionSide": "SHORT",
#     "status": "NEW",
#     "stopPrice": "9300",        // please ignore when order type is TRAILING_STOP_MARKET
#     "closePosition": false,   // if Close-All
#     "symbol": "BTCUSDT",
#     "timeInForce": "GTC",
#     "type": "TRAILING_STOP_MARKET",
#     "origType": "TRAILING_STOP_MARKET",
#     "activatePrice": "9020",    // activation price, only return with TRAILING_STOP_MARKET order
#     "priceRate": "0.3",         // callback rate, only return with TRAILING_STOP_MARKET order
#     "updateTime": 1566818724722,
#     "workingType": "CONTRACT_PRICE",
#     "priceProtect": false            // if conditional order trigger is protected   
# }

        broker_mapping ={ 'order_types': { bt.Order.Market: 'market', bt.Order.Limit: 'limit', bt.Order.Stop: 'stop-loss', bt.Order.StopLimit: 'STOP_LOSS_LIMIT',
                                          bt.Order.StopTrail:'TRAILING_STOP_MARKET'},#bt.Order.StopTrailLimit:''}, 
         'mappings': {'closed_order': {'key': 'status','value': 'closed'},'canceled_order': {'key': 'status','value': 'canceled' } } }
        hist_start_date = dt.utcnow() - timedelta(minutes=LAGS*MINUTES)
        data = store.getdata( dataname=f'{COIN_TARGET}/{COIN_REFER}' , name=f'{COIN_TARGET}/{COIN_REFER}', timeframe=bt.TimeFrame.Minutes,fromdate=hist_start_date, compression=MINUTES, ohlcv_limit=LAGS*MINUTES, drop_newest=True )
        cerebro.adddata(data)#,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        broker = store.getbroker(broker_mapping=broker_mapping)
        print('pos:',broker.getposition(data).size)
        print('possize',broker.positions['BTC/USDT'].size)
        print('balance:',broker.get_balance())
        #print(f'orders open{broker.get_orders_open(symbol="BTCUSDT")}')
        print('open orders:',broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        print(broker.private_end_point(type='GET',endpoint='/openOrders',params={'symbol':"BTCUSDT"}))
        cerebro.setbroker(broker)
        #data1=KDBFeed(sym='BTCUSDT@binance',freq="1Min",table='pptliqs',fromdate=None,todate=None,addwhere=', side=`sell , vol>0.01',timeframe=bt.TimeFrame.Minutes ,compression=1)
        #cerebro.adddata(data1)
        #add low freq, add preload for one year from hdb
        #data2=KDBFeed(sym='BTCUSDT@binance',freq="5Min",table='pptliqs',fromdate=None,todate=None,addwhere=', side=`sell , vol>0.01',timeframe=bt.TimeFrame.Minutes ,compression=5)
        #cerebro.adddata(data2)
    elif BACKTEST:
        cerebro.addobserver(bt.observers.Value)
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
        cerebro.addanalyzer(trade_list,_name='tradelist')
        cerebro.addanalyzer(bt.analyzers.Transactions)
        dfbtc=qbars('BTCUSDT@binance',freq=str(MINUTES)+"Min",table='ppttrades',date=[date(2021,7,29),date(2021,8,4)],addwhere=', vol>0.01')
        #dfbtc=qbars('BTCUSDT_210924@binance',freq=str(MINUTES)+"Min",table='ppttrades',date=[date(2021,8,1),date(2021,8,1)],addwhere=', vol>0.01')
        
        dfbtc1=dfbtc#.loc['2021-05']
        data=bt.feeds.PandasData(dataname=dfbtc1.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None,timeframe=bt.TimeFrame.Minutes,compression=MINUTES)
        cerebro.adddata(data)
        #cerbero.adddata(qbars())
        
    logging.critical("alfabot USDCUSDT cerebro.run()")
    backtest = cerebro.run(runonce=False) if not BACKTEST else cerebro.run(runonce=False,tradehistory=True)
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % final_value)
    if BACKTEST:
        rcsave=plt.rcParams['figure.figsize']
        plt.rcParams['figure.figsize']=[30, 16]
        cerebro.plot(style='line', rowsmajor=1, rowsminor=1,volume=True, iplot=False, barup='green', bardown='grey',fmt_x_ticks = '%Y-%b-%d %H:%M',fmt_x_data = '%Y-%b-%d %H:%M')
        plt.rcParams['figure.figsize']=rcsave
        
        btres = backtest[0]
        for k in ['total','pnl']:#backtest[0].analyzers.ta.get_analysis():
            print (k,dict(backtest[0].analyzers.ta.get_analysis()[k]))
        #print(dict(backtest[0].analyzers.sharperatio.get_analysis()))

        printTradeAnalysis(cerebro, backtest[0].analyzers)
        print("amount,price,sid,symbol,value")
        display(DF(btres.analyzers.tradelist.get_analysis()).set_index('datein'))
        
try:
    main()
except KeyboardInterrupt:
    print("finished.")

In [ ]:
1

In [ ]:
srt=get_secret('b1-wzxNSw')
broker_config = {'apiKey': srt['k'],'secret': srt['s'],'nonce': lambda: str(int(time.time() * 1000)),'enableRateLimit': True, }
broker_config['options']={'defaultType': 'future' }#if 1==1 else 'spot'}
store = CCXTStore(exchange='binance', currency='USDT', config=broker_config, retries=5, debug=True)
broker = store.getbroker()
display(DF(broker.store.exchange.fetch_my_trades(symbol="BTC/USDT")).set_index('datetime').gr())
display(DF(broker.store.exchange.fetch_orders(symbol="BTC/USDT")).gr())

In [ ]:
freq=str(MINUTES)+"Min"

In [ ]:
dfbtc=qbars('BTCUSDT_210924@binance',freq="1Min",table='ppttrades',date=[date(2021,8,1),date(2021,8,5)],addwhere=', vol>0.01').loc['2021-08-01 23:00':]
df5m=qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[date(2021,8,1),date(2021,8,5)],addwhere=', side=`sell , vol>0.01',verbose=0).loc['2021-08-01 23:00':]        

In [ ]:
dfbtc

In [ ]:
def trailingstop(df,entries,trailingperc):
    exits=entries*False
    
    df['ts']=df['c'].cummax()*(1-trailingperc)
    df['exit']=df['c']<df['ts']
    df.iplot()
    return df['exit']
    
    #return exits
trailingstop(dfbtc,res['entries_dirty'],0.02)[trailingstop(dfbtc,res['entries_dirty'],0.02)]
    
    

In [ ]:
def mybttrailing(df0,df1,lag=20,liqlimit=10,ts_stop=2,stratfreq="5Min",verbose=0,plot=False):
    entries=[]
    exits=[]
    
    dirtylongs=df1[df1['v']>liqlimit].index+pd.Timedelta(stratfreq)
    df=DF(dirtylongs,index=dirtylongs)
    df.columns=['t1'] #t1 is entry
    df['dt']=df.t1.diff(1)
    
    df['t2']=np.nan
#     print(f"de{dirtylongs}")
#     print(f"iff dl{dirtylongs.to_series().diff(1)}")
    
    for i,r in df.iterrows():
 #       print(f"r t1={r['t1']}")
        t2=(df0['c'].loc[df0.index>r['t1']]<df0['c'].loc[df0.index>r['t1']].cummax()*(1-ts_stop/100))#.index
        t2=t2[t2]
        if len(t2)==0:
            break
        #display(t2)
        df.loc[r['t1'],'t2']=t2.index[0]
        #display(df)
#    display(df)
        
    df['ok']=1
    #ipdb.set_trace()
    curt2=df['t2'].iloc[0]
    dfiter=df.iterrows()
    next(dfiter)
    
    for i,r in dfiter:
#        print(f"i={i} {r['t1']}<{curt2}    isnan {pd.isna(curt2)}")
        
        if r['t1']<curt2:
            df['ok'].loc[i]=0
        else:
            
            if pd.isna(r['t2']):
                #fill the rest of still open positions to 0
                df['ok'].loc[curt2:]=0
                break
            else:
                curt2=r['t2']
    df=df[df.ok==1]
#    display(df)

    entries_clean=pd.Series(True,index=df['t1'])
    exits_clean=pd.Series(True,index=df['t2'])
    entries= pd.Series.vbt.signals.empty_like(df0['c'])
    exits= pd.Series.vbt.signals.empty_like(df0['c'])
    entries.update(entries_clean)#=True
    exits.update(exits_clean)
    #exits[exits_clean]=True
    entries=entries.astype(bool)
    exits=exits.astype(bool)
#    display(entries)
    
#     entries_dirty.loc[df1[df1['v']>liqlimit].index+pd.Timedelta("5Min")]=True
#     entries_dirty.fillna(False,inplace=True)
#     display(entries_dirty[entries_dirty])
    #exits_dirty = vbt.OHLCSTEX.run(entries_dirty,df0['o'],df0['h'],df0['l'],df0['c'],ts_stop=ts_stop/100, wait=0,stop_type=None, hit_price=None).exits
    #exits_dirty=vbt.OHLCSTX.run(entries_dirty,df0['o'],df0['h'],df0['l'],df0['c'],sl_trail=True,sl_stop=ts_stop/100,stop_type=1,stop_price=None).exits
    #class OHLCSTCX wrapper, input_list, input_mapper, in_output_list, output_list, param_list, mapper_list, short_name, level_names
    #def run #entries, open, high, low, close, sl_stop=Default(nan), sl_trail=Default(False), tp_stop=Default(nan), stop_price=nan, stop_type=-1, short_name='ohlcstcx', hide_params=None, hide_default=True, **kwargs)
#    display(exits_dirty[exits_dirty])
#    entries_dirty = df0['spread'] < bb['bbl.spread']
#    exits_dirty = bb['spread'] > bb['bbh.spread']
    #entries[entries==True]
#    entries,exits = pd.Series.vbt.signals.clean(entries_dirty,exits_dirty, entry_first=True, broadcast_kwargs=dict(columns_from='keep'))
    if plot:
        fig = vbt.make_subplots(specs=[[{"secondary_y": True}]])
        fig = df0['c'].vbt.plot(trace_kwargs=dict(name='df0.c'), fig=fig)
        #fig = df1['c'].vbt.plot(trace_kwargs=dict(name='df1.c'), fig=fig)
#        print(f"entries:{type(entries)} exits:{type(exits)}")
        display(entries.head())
        display(exits.head())
        fig = entries.vbt.signals.plot_as_entry_markers(df0['c'], fig=fig)
        fig = exits.vbt.signals.plot_as_exit_markers(df0['c'], fig=fig)
        fig.update_layout(template='vbt_dark', showlegend=False, width=1600, height=300)
        fig.show()
    # #ts exits
    # # Execute at the next bar
    # vbt_entries= pd.Series.vbt.signals.empty_like(dfbtc1['c'])
    # vbt_entries.loc[dfliq1[dfliq1['v'].shift(1)>liqlimit].index]=True
    # ts_exits = vbt.OHLCSTEX.run(vbt_entries,df0['o'],df0['h'],dfbtc1['l'],dfbtc1['c'],ts_stop=list(stops),wait=1,stop_type=None, hit_price=None).exits
    # vbt_entries, vbt_exits = pd.DataFrame.vbt.signals.clean(vbt_entries, ts_exits)
    # portf = vbt.Portfolio.from_signals(dfbtc1['o'], vbt_entries, ts_exits, price=dfbtc1['o'])#.vbt.fshift(1))
    #percentage order size
    symbol_cols = pd.Index([0, 1], name='symbol')
    ordersize = pd.DataFrame(index=entries.index)#, columns=symbol_cols)
    ordersize[0] = np.nan
    #ordersize[1] = np.nan
    #ordersize.loc[exits] = -0.001
    ordersize.loc[entries,0] = 0.001

    #shoft ordersize
    #ordersize = ordersize.vbt.fshift(1)
    
    portf = vbt.Portfolio.from_signals(df0['c'], entries, exits, price=df0['o'],size=0.001,freq='1Min')#.vbt.fshift(1))
    
#     portf=vbt.Portfolio.from_orders(
#             close=df0['c'], # current close as reference price
#             size=ordersize, 
#             price=df0['o'], # current open as execution price
#             size_type='targetpercent',
#             val_price=df0['c'],  # previous close as group valuation price
#             init_cash=10000,
#             fees=0.0006,
#             cash_sharing=True,  # share capital between assets in the same group
#             group_by=True,  # all columns belong to the same group
#             call_seq='auto',  # sell before buying
#             freq='d'  # index frequency for annualization
#         )

    if verbose:
        entries.astype('float').plot()
        exits.astype('float').plot()
        #ordersize.dropna().plot(marker='x')
        display(portf.orders.records_readable)
        display(portf.trades.records_readable[::-1])
        print(f"mean PNL={portf.trades.records_readable['PnL'].describe()}")
        print(f"mean return PNL={portf.trades.records_readable['Return'].describe()}")
        
        
        #print(portf.stats())
    return {'portf':portf,'entries':entries,'exits':exits,'entries_dirty':entries,'exits_dirty':exits,'SR':portf.sharpe_ratio()}
# dfbtc=qbars('BTCUSDT@binance',freq="1Min",table='ppttrades',date=[date(2020,1,1),date(2021,8,1)],addwhere=', vol>0.01')#.loc['2021-08-01 23:00':]
# df5m=qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[date(2020,1,1),date(2021,8,1)],addwhere=', side=`sell , vol>0.01',verbose=0)#.loc['2021-08-01 23:00':]    
dfbtc=qbars('BTCUSDT@binance',freq="1Min",table='ppttrades',date=[date(2021,8,1),date(2021,8,5)],addwhere=', vol>0.01').loc['2021-08-01 23:00':]
df5m=qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[date(2021,8,1),date(2021,8,5)],addwhere=', side=`sell , vol>0.01',verbose=0).loc['2021-08-01 23:00':]    

mybttrailing(dfbtc,df5m,lag=20,ts_stop=2,verbose=1,plot=False)

In [ ]:
dff=pd.DataFrame(columns=pd.MultiIndex.from_tuples([(np.nan,np.nan)],names=['trailpercent','inflowlimit']),index=dfbtc.index)
dff.head(2)

resSRtotal=[]
resNTtotal=[]


# return {'portf':portf,'entries':entries,'exits':exits,'entries_dirty':entries,'exits_dirty':exits,'SR':portf.sharpe_ratio()}
# mybttrailing(dfbtc,df5m,lag=20,ts_stop=2,verbose=0,plot=False)


for trailpercent in np.linspace(1,7,20):
    for inflowlimit in np.linspace(1,30,30):
        #stratres=btstrat(lags=100,trailpercent=trailpercent,inflowlimit=inflowlimit,plot=False)
        stratres=mybttrailing(dfbtc,df5m,lag=20,liqlimit=inflowlimit,ts_stop=trailpercent,verbose=0,plot=False)   #def mybttrailing(df0,df1,lag=20,liqlimit=10,ts_stop=2,stratfreq="5Min",verbose=0,plot=False):
        print({'trailpercent':trailpercent,'inflowlimit':inflowlimit,'SR':stratres['SR']})
        #dff[trailpercent,inflowlimit]=stratres['portf'].['rets']#.copy()
        resSRtotal.append({'trailpercent':trailpercent,'inflowlimit':inflowlimit,'SR':stratres['SR']})
        #resNTtotal.append({'trailpercent':trailpercent,'inflowlimit':inflowlimit,'NT':stratres['nt']/((dff.index[-1]-dff.index[0])/pd.Timedelta('4.33w'))

        


plt.rcParams['figure.figsize'] = (20.0, 20.0)
DF(resSRtotal).pivot('trailpercent','inflowlimit').heatmap()
plt.show()
dff.drop(np.nan, axis=1, level=1,inplace=True)
dff.head()     
DF(resNTtotal).pivot('trailpercent','inflowlimit').heatmap()#figsize=(20,20))#cmap='RdYlGn',fmt='.3g',annot=True)
dff.dropna().cumsum().iplot()
#possible that all the N strategy configs have high but similar Sharpe ratios. Therefore, PBO may appear high, however, 'overfitting' here is among many 'skilful' strategies.
#pypbo.pbo(dff.dropna(), S=8, metric_func=lambda x: perf.sharpe_iid(x), threshold=1, n_jobs=4,plot=True, verbose=False, hist=False)
#pbo stability
pbos=[]
for S in [4,6,8,10,12,14]:
    pbos.append({"S":S,'pbo':pypbo.pbo(dff.dropna(), S=S, metric_func=lambda x: perf.sharpe_iid(x), threshold=1, n_jobs=4,plot=False, verbose=False, hist=False).pbo})
DF(pbos).set_index('S').plot()

In [ ]:
mybttrailing(dfbtc,df5m,lag=20,liqlimit=1,ts_stop=1,verbose=0,plot=True) 

In [ ]:
import vectorbt as vbt
def vbtstrat(df0,df1,lag=20,liqlimit=10,ts_stop=2,nstdl=2,nstdh=2,methodma='ma',methodstd='ma',verbose=0,plot=False):
    #entries_dirty= pd.Series.vbt.signals.empty_like(DF(index=df0['c'].index))
    entries_dirty= pd.Series.vbt.signals.empty_like(df0['c'])
    entries_dirty.loc[df1[df1['v']>liqlimit].index+pd.Timedelta("5Min")]=True
    #strategy
    #dfentr=DF(df1[['v']].shift(1).values>liqlimit,index=df1['v'].index)
    #entries_dirty.update(dfentr)
    entries_dirty.fillna(False,inplace=True)
    display(entries_dirty[entries_dirty])


    #exits_dirty = vbt.OHLCSTEX.run(entries_dirty,df0['o'],df0['h'],df0['l'],df0['c'],ts_stop=ts_stop/100, wait=0,stop_type=None, hit_price=None).exits
    exits_dirty=vbt.OHLCSTX.run(entries_dirty,df0['o'],df0['h'],df0['l'],df0['c'],sl_trail=True,sl_stop=ts_stop/100,stop_type=1,stop_price=None).exits
    
    #class OHLCSTCX wrapper, input_list, input_mapper, in_output_list, output_list, param_list, mapper_list, short_name, level_names
    
    #def run #entries, open, high, low, close, sl_stop=Default(nan), sl_trail=Default(False), tp_stop=Default(nan), stop_price=nan, stop_type=-1, short_name='ohlcstcx', hide_params=None, hide_default=True, **kwargs)
    
    
    
    
    display(exits_dirty[exits_dirty])
#    entries_dirty = df0['spread'] < bb['bbl.spread']
#    exits_dirty = bb['spread'] > bb['bbh.spread']
    #entries[entries==True]
    entries,exits = pd.Series.vbt.signals.clean(entries_dirty,exits_dirty, entry_first=True, broadcast_kwargs=dict(columns_from='keep'))
    
    if plot:
        fig = vbt.make_subplots(specs=[[{"secondary_y": True}]])
        fig = df0['c'].vbt.plot(trace_kwargs=dict(name='df0.c'), fig=fig)
        #fig = df1['c'].vbt.plot(trace_kwargs=dict(name='df1.c'), fig=fig)
        print(f"entries:{type(entries)} exits:{type(exits)}")
        display(entries.head())
        display(exits.head())
        fig = entries.vbt.signals.plot_as_entry_markers(df0['c'], fig=fig)
        fig = exits.vbt.signals.plot_as_exit_markers(df0['c'], fig=fig)
        fig.update_layout(template='vbt_dark', showlegend=False, width=1600, height=300)
        fig.show()
    
    # #ts exits
    # # Execute at the next bar
    # vbt_entries= pd.Series.vbt.signals.empty_like(dfbtc1['c'])
    # vbt_entries.loc[dfliq1[dfliq1['v'].shift(1)>liqlimit].index]=True
    # ts_exits = vbt.OHLCSTEX.run(vbt_entries,df0['o'],df0['h'],dfbtc1['l'],dfbtc1['c'],ts_stop=list(stops),wait=1,stop_type=None, hit_price=None).exits
    # vbt_entries, vbt_exits = pd.DataFrame.vbt.signals.clean(vbt_entries, ts_exits)
    # portf = vbt.Portfolio.from_signals(dfbtc1['o'], vbt_entries, ts_exits, price=dfbtc1['o'])#.vbt.fshift(1))
    #percentage order size
    symbol_cols = pd.Index([0, 1], name='symbol')
    ordersize = pd.DataFrame(index=entries.index)#, columns=symbol_cols)
    ordersize[0] = np.nan
    #ordersize[1] = np.nan
    #ordersize.loc[exits] = -0.001
    ordersize.loc[entries,0] = 0.001

    #shoft ordersize
    #ordersize = ordersize.vbt.fshift(1)
    
    portf = vbt.Portfolio.from_signals(df0['c'], entries, exits, price=df0['o'],size=0.001,freq='1Min')#.vbt.fshift(1))
    
#     portf=vbt.Portfolio.from_orders(
#             close=df0['c'], # current close as reference price
#             size=ordersize, 
#             price=df0['o'], # current open as execution price
#             size_type='targetpercent',
#             val_price=df0['c'],  # previous close as group valuation price
#             init_cash=10000,
#             fees=0.0006,
#             cash_sharing=True,  # share capital between assets in the same group
#             group_by=True,  # all columns belong to the same group
#             call_seq='auto',  # sell before buying
#             freq='d'  # index frequency for annualization
#         )

    if verbose:
        entries.astype('float').plot()
        exits.astype('float').plot()
        #ordersize.dropna().plot(marker='x')
        display(portf.orders.records_readable)
        display(portf.trades.records_readable[::-1])
        #print(portf.stats())
    return {'portf':portf,'entries':entries,'exits':exits,'entries_dirty':entries_dirty,'exits_dirty':exits_dirty,'SR':portf.sharpe_ratio()}
res=vbtstrat(dfbtc,df5m,lag=20,ts_stop=2,nstdl=2,nstdh=2,methodma='ma',methodstd='ma',verbose=1,plot=True)

In [ ]:
res['exits_dirty'][res['exits_dirty']]

In [ ]:
res['entries_dirty'][res['entries_dirty']]

In [ ]:
dfbtc.loc['2021-08-03 18:10:00':'2021-08-03 18:11:00']    

In [ ]:
dfbtc.loc['2021-08-04 07:05:05':'2021-08-05 23:59:05'].assign(ts2=lambda x:x['c'].cummax()*0.98).iplot()

In [ ]:
entries_dirty= pd.Series.vbt.signals.empty_like(dfbtc['c'])
entries_dirty.loc[df5m[df5m['v'].shift(1)>10].index]=True
entries_dirty[entries_dirty]

In [ ]:
res['entries'][res['entries']]

In [ ]:
l=ss.split('\n')
l2=[]
for l1 in l:
    if len(l1)>7 and (l1[0]!='0' and l1[0]!='3' and 'query' not in l1[:min(len(l1),7)]):
        l2.append(l1)
print('\n'.join(l2))